In [1]:
import torch
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer, pipeline, CLIPProcessor, CLIPModel
from PIL import Image
import pandas as pd
import requests

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
r = requests.get('https://sigongan-3f44b-default-rtdb.firebaseio.com/archive.json').json()
list = [value for key, value in r.items()]
df = pd.DataFrame(list)

In [3]:
from g_spread import *
gsp = GSpread()

In [ ]:
gsp.updateByDf(df[['imageUrl', 'description']], [4,2])

In [ ]:
image_to_text = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")
df['captioning'] = df.apply(lambda x: image_to_text(x.imageUrl)[0]['generated_text'], axis = 1)

In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

In [5]:
def classifier1(url, label=['real photo', 'document']):
    image = Image.open(requests.get(url, stream=True).raw)
    inputs = processor(text=label, images=image, return_tensors="pt", padding=True)
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image # this is the image-text similarity score
    probs = logits_per_image.softmax(dim=1) #
    return torch.max(probs, dim=1).indices.item()

In [8]:
df['class1'] = df.imageUrl.apply(lambda x: classifier1(x, label=['picture', 'document']))

In [10]:
gsp.updateByDf(df[['class1']], [4, 4])